In [1]:
import pandas as pd
import numpy as np
import random
from tqdm import tqdm
from gensim.models import Word2Vec 
import matplotlib.pyplot as plt
%matplotlib inline

import warnings;
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('./fashion_wear.csv')

In [3]:
df.head()

,Unnamed: 0,user_ID,user_Fans,outfit_ID,outfit_Url,outfit_Like,outfit_Like_adjust,outfit_Update,outfit_Season,color,outfit_Bodypart,outfit_Itemgroup,outfit_Item
0,0,ass03,170570,18406885,https://wear.tw/ass03/18406885/,277,277.0,2021-01-03,winter,白色,outerwear,夾克／外套,白色西裝大衣
1,1,ass03,170570,18406885,https://wear.tw/ass03/18406885/,277,277.0,2021-01-03,winter,粉色,acc,時尚配件,粉色圍巾
2,2,ass03,170570,18406885,https://wear.tw/ass03/18406885/,277,277.0,2021-01-03,winter,黑色,acc,包,黑色包
3,3,ass03,170570,18406885,https://wear.tw/ass03/18406885/,277,277.0,2021-01-03,winter,黑色,acc,鞋子,黑色靴子
4,4,ass03,170570,18395363,https://wear.tw/ass03/18395363/,282,282.0,2020-12-31,winter,綠色,outerwear,夾克／外套,綠色夾克／外套


In [4]:
df.shape

(345511, 13)

In [5]:
#check null value
df.isnull().sum()

Unnamed: 0                0
user_ID                   0
user_Fans                 0
outfit_ID                 0
outfit_Url                0
outfit_Like               0
outfit_Like_adjust        0
outfit_Update             0
outfit_Season             0
color                 12730
outfit_Bodypart           0
outfit_Itemgroup          0
outfit_Item              11
dtype: int64

In [6]:
df.dropna(axis = 0, how ='any',inplace=True) 
df.isnull().sum()

Unnamed: 0            0
user_ID               0
user_Fans             0
outfit_ID             0
outfit_Url            0
outfit_Like           0
outfit_Like_adjust    0
outfit_Update         0
outfit_Season         0
color                 0
outfit_Bodypart       0
outfit_Itemgroup      0
outfit_Item           0
dtype: int64

In [7]:
df.shape

(332781, 13)

In [8]:
users = df["user_ID"].unique().tolist()
len(users)

99

In [9]:
print(users)

['ass03', 'miyoshi0511', 'takahashiai', 'saesan1201', 'choco24', 'moken', 'lily02tbt', '34cm', 'moimaru6', 'suzuchan1206', 'unitarosu9876', 'harupi1230', 'pageboy_hinechi', 'haushinka71', 'fwafwa7', 'mariko0121', 'akari0302', 'chicchimo5', 'mitsueri', 'riho0914', 'aobaeru', 'maamin0922', 'natsu0315', 'yuriaaa5', 'jeaaami', 'taroumaru41', 'marumaru1530', 'shino1226', 'yunpi', '2007330', 'mari0109', 'na56482n', 'nananeayane', '61x35', 'zipumi', 'xxyukie522xx', 'enosaya', 'mhpht555', 'sanki0102', 'cloud9nail', 'mii207', '0510a', 'yoahiru35', 'a03man', 'kooooomi64', 'tera1008', 'jinjin1018', 'nanamiii11', 'poconyanco', 'an3chan', 'sonomi0610', 'imoko30', 'pjtamug', '0627himari', 'yukieiura2015', 'kanoco', 'pyon94', 'ys17', '12chi24', 'lvexxxx', 'cozue52', '0202hn', '38hnt', 'miniwen311', 'rinaty1', 'murasesae', 'ayuaum', 'eerrkkk', 'cotara7', 'chorochoro10', 'kco3211', 'mefil', 'unakmkmay', '3186mm', 'carmen1505', 'ayacyocopiiiman', 'aaaannnn0726', 'mmmmmtmt', 'aya35s', 'uethrnrk', 'merore

In [10]:
df['outfit_Item']= df['outfit_Item'].astype(str)

In [11]:
#split data into train and validation dataset according to the upload dataset(history) for each model
random.shuffle(users)
users_train = [users[i] for i in range(round(0.8*len(users)))] 
train_df = df[df['user_ID'].isin(users_train)]
validation_df = df[~df['user_ID'].isin(users_train)]

In [12]:
# list to capture upload history of the users
upload_train = []
for i in tqdm(users_train):
    temp = train_df[train_df["user_ID"] == i]["outfit_Item"].tolist()
    upload_train.append(temp)
print(upload_train[0])

100%|██████████| 99/99 [00:01<00:00, 59.75it/s]

['白色連帽外套', '棕色插肩外套', '黑色靴子', '白色連帽外套', '黑色靴子', '藍色運動衫', '綠色褲裝', '黑色靴子', '白色針織衫', '白色針織衫', '米色插肩外套', '黑色靴子', '白色針織衫', '黑色褲裝', '黑色靴子', '灰色襯衫', '灰色短外套', '黑色靴子', '黑色針織衫', '棕色短外套', '棕色褲裝', '米色短外套', '白色針織衫', '黑色褲裝', '黑色靴子', '黑色針織衫', '米色短外套', '白色T恤', '棕色褲裝', '藍色襯衫', '黑色靴子', '藍色襯衫', '黑色短外套', '黑色靴子', '紅色襯衫', '黑色靴子', '紅色襯衫', '棕色短外套', '黑色靴子', '灰色襯衫', '黑色靴子', '米色運動衫', '米色褲裝', '黑色靴子', '米色運動衫', '米色短外套', '黑色靴子', '米色褲裝', '棕色短外套', '藍色針織衫', '棕色褲裝', '黑色靴子', '米色褲裝', '白色針織衫', '黑色靴子', '米色短外套', '黑色靴子', '米色褲裝', '藍色針織衫', '綠色羽絨衣/大衣', '棕色褲裝', '黑色靴子', '黑色褲裝', '棕色針織衫', '黑色靴子', '白色T恤', '黑色褲裝', '黑色靴子', '黑色T恤', '黑色褲裝', '黑色靴子', '黑色西裝大衣', '黑色褲裝', '黑色靴子', '黑色T恤', '白色T恤', '黑色褲裝', '黑色靴子', '棕色西裝大衣', '白色運動衫', '黑色褲裝', '黑色靴子', '黑色短外套', '白色運動衫', '黑色褲裝', '黑色靴子', '紫色針織衫', '黑色靴子', '黑色襯衫', '黑色靴子', '黑色插肩外套', '米色T恤', '藍色褲裝', '黑色靴子', '藍色褲裝', '米色T恤', '黑色靴子', '黑色褲裝', '黑色靴子', '白色羽絨衣/大衣', '黑色褲裝', '米色插肩外套', '黑色靴子', '黑色靴子', '黑色針織衫', '黑色襯衫', '紫色針織衫', '黑色羽絨衣/大衣', '黑色靴子', '黑色短外套', '黑色褲裝', '黑色靴子', '銀色項鏈', '白色褲裝', '白色襯衫', '黑色靴子', '棕色短外套', '黑色襯衫

In [13]:
upload_val = []
for i in tqdm(validation_df['user_ID'].unique()):
    temp = validation_df[validation_df["user_ID"] == i]["outfit_Item"].tolist()
    upload_val.append(temp)

In [14]:
# train word2vec model
model = Word2Vec(window = 2, sg = 1, hs = 0, #window=2
                 negative = 10, # for negative sampling
                 alpha=0.03, min_alpha=0.0007,
                 seed = 14)

model.build_vocab(upload_train, progress_per=200)

model.train(upload_train, total_examples = model.corpus_count, 
            epochs=10, report_delay=1)

(2221570, 3327810)

In [15]:
model.save("word2vec_wear.model")

In [16]:
print(model)

Word2Vec(vocab=1089, size=100, alpha=0.03)


In [17]:
# extract all vectors
X = model[model.wv.vocab]
X.shape

(1089, 100)

In [20]:
#remove duplicates
outfit_Url=train_df['outfit_Url']
outfit_Url.drop_duplicates(inplace=True, keep="last")
outfit_Url.shape

(82324,)

In [23]:
# create product-ID and product-description dictionary
outfit_info = train_df[["outfit_Item", "outfit_Url","outfit_Like_adjust","outfit_Season","outfit_Bodypart"]]
outfit_info.sort_values(by=['outfit_Like_adjust'], inplace=True,ascending=False)
outfit_dict = outfit_info.groupby(['outfit_Season','outfit_Item'])['outfit_Url'].apply(list).to_dict()

In [39]:
# Make an Recommendatin with most similar goods 
def similar_products(season,v, n = 4):
    m =  model.similar_by_vector(v, topn= 4)[0]
    m_recommend_outfitUrl=(m[0],outfit_dict[season,m[0]][:3] , m[1])
    
    # extract most similar products for the input vector
    ms = model.similar_by_vector(v, topn= 10)[1:]
    ms_recommend_outfitUrl = []
    for j in ms:   
        try:
            pair = (j[0],outfit_dict[season,j[0]][:3] , j[1])
            ms_recommend_outfitUrl.append(pair)
        except:
            continue
    return m_recommend_outfitUrl, ms_recommend_outfitUrl

In [40]:
similar_products('winter',model['米色休閒長褲'])

(('米色休閒長褲',
  ['https://wear.tw/maamin0922/18313109/',
   'https://wear.tw/ass03/18363523/',
   'https://wear.tw/yuumi557188/16211264/'],
  1.0),
 [('藍色休閒長褲',
   ['https://wear.tw/yuumi557188/8778822/',
    'https://wear.tw/takahashiai/8733349/',
    'https://wear.tw/yoahiru35/11613161/'],
   0.6507023572921753),
  ('米色休閒短褲',
   ['https://wear.tw/ayacyocopiiiman/13764468/'],
   0.6204767227172852),
  ('藍色牛仔褲',
   ['https://wear.tw/chicchimo5/16136605/',
    'https://wear.tw/unitarosu9876/16036107/',
    'https://wear.tw/choco24/9280432/'],
   0.6157659292221069),
  ('黑色休閒長褲',
   ['https://wear.tw/choco24/13993775/',
    'https://wear.tw/riho0914/18246076/',
    'https://wear.tw/mhpht555/18198087/'],
   0.6138306856155396),
  ('棕色連體褲/背帶褲',
   ['https://wear.tw/unitarosu9876/16174669/',
    'https://wear.tw/unitarosu9876/16215954/',
    'https://wear.tw/chicchimo5/15822166/'],
   0.5919194221496582),
  ('綠色牛仔褲',
   ['https://wear.tw/enosaya/18280371/',
    'https://wear.tw/dossii/1823920

In [49]:
# Make an Recommendatin for fashion ensemble
def similar_products_byitem(season,v, n = 4):
    m =  model.similar_by_vector(v, topn= 4)[0]
    m_recommend_outfitUrl=(m[0],outfit_dict[season,m[0]][:3] , m[1])
    
    # extract most similar products for the input vector of different product categories
    ms = model.similar_by_vector(v, topn= 100)[1:]
    ms_recommend_outfitUrl = []
    for j in ms:
        if outfit_info['outfit_Bodypart'][outfit_info['outfit_Item']==m[0]].values[0]=='top':
            if outfit_info['outfit_Bodypart'][outfit_info['outfit_Item']==j[0]].values[0] \
            in ['outerear','buttom']:
                try:
                    pair = (j[0],outfit_dict['summer',j[0]][:3] , j[1])
                    ms_recommend_outfitUrl.append(pair)
                except:
                    continue
        elif outfit_info['outfit_Bodypart'][outfit_info['outfit_Item']==m[0]].values[0]=='buttom':
            if outfit_info['outfit_Bodypart'][outfit_info['outfit_Item']==j[0]].values[0] \
            in ['top','outerwear']:
                try:
                    pair = (j[0],outfit_dict['summer',j[0]][:3] , j[1])
                    ms_recommend_outfitUrl.append(pair)
                except:
                    continue
        elif outfit_info['outfit_Bodypart'][outfit_info['outfit_Item']==m[0]].values[0]=='outerwear':
            if outfit_info['outfit_Bodypart'][outfit_info['outfit_Item']==j[0]].values[0] \
            in ['top','buttom','dress']:
                try:
                    pair = (j[0],outfit_dict['summer',j[0]][:3] , j[1])
                    ms_recommend_outfitUrl.append(pair)
                except:
                    continue
        elif outfit_info['outfit_Bodypart'][outfit_info['outfit_Item']==m[0]].values[0]=='dress':
            if outfit_info['outfit_Bodypart'][outfit_info['outfit_Item']==j[0]].values[0] \
            in ['outerwear']:
                try:
                    pair = (j[0],outfit_dict['summer',j[0]][:3] , j[1])
                    ms_recommend_outfitUrl.append(pair)   
                except:
                    continue
    return m_recommend_outfitUrl, ms_recommend_outfitUrl

In [50]:
similar_products_byitem('summer',model['白色褲裝'])

(('白色褲裝',
  ['https://wear.tw/yoahiru35/17301668/',
   'https://wear.tw/fwafwa7/17455693/',
   'https://wear.tw/maarimo196/17383511/'],
  1.0),
 [('黃色背心外套', ['https://wear.tw/maarimo196/17265532/'], 0.595501720905304),
  ('橘色套裝',
   ['https://wear.tw/ninomasa/17527184/',
    'https://wear.tw/ninomasa/17350060/'],
   0.5745309591293335),
  ('米色披肩',
   ['https://wear.tw/carmen1505/15176425/',
    'https://wear.tw/carmen1505/14977728/'],
   0.5720653533935547),
  ('白色套裝',
   ['https://wear.tw/chicchimo5/17126803/',
    'https://wear.tw/akari0302/17478080/',
    'https://wear.tw/chicchimo5/14983703/'],
   0.5645591020584106),
  ('米色連體套裝',
   ['https://wear.tw/sanki0102/17016617/',
    'https://wear.tw/cloud9nail/12556345/'],
   0.5632511377334595),
  ('紅色背心外套',
   ['https://wear.tw/yuriaaa5/17367330/',
    'https://wear.tw/ninomasa/14956278/'],
   0.5563393831253052),
  ('米色連帽長外套',
   ['https://wear.tw/keiru2000/17477143/',
    'https://wear.tw/mii207/17477758/',
    'https://wear.tw/harup

In [41]:
#Make an Recommendatin for fashion ensemble(Accessory)
def similar_products_byitem_acc(season,v, n = 4):
    m =  model.similar_by_vector(v, topn= 4)[0]
    m_recommend_outfitUrl=(m[0],outfit_dict[season,m[0]][:3] , m[1])
    
    # extract most similar accessories for the input vector 
    ms = model.similar_by_vector(v, topn= 50)[1:]
    ms_recommend_outfitUrl = []
    for j in ms:
        if outfit_info['outfit_Bodypart'][outfit_info['outfit_Item']==m[0]].values[0]=='top':
            if outfit_info['outfit_Bodypart'][outfit_info['outfit_Item']==j[0]].values[0] \
            in ['outerear','buttom','acc']:
                try:
                    pair = (j[0],outfit_dict['summer',j[0]][:3] , j[1])
                    ms_recommend_outfitUrl.append(pair)
                except:
                    continue
        elif outfit_info['outfit_Bodypart'][outfit_info['outfit_Item']==m[0]].values[0]=='buttom':
            if outfit_info['outfit_Bodypart'][outfit_info['outfit_Item']==j[0]].values[0] \
            in ['top','outerwear','acc']:
                try:
                    pair = (j[0],outfit_dict['summer',j[0]][:3] , j[1])
                    ms_recommend_outfitUrl.append(pair)
                except:
                    continue
        elif outfit_info['outfit_Bodypart'][outfit_info['outfit_Item']==m[0]].values[0]=='outerwear':
            if outfit_info['outfit_Bodypart'][outfit_info['outfit_Item']==j[0]].values[0] \
            in ['top','buttom','dress','acc']:
                try:
                    pair = (j[0],outfit_dict['summer',j[0]][:3] , j[1])
                    ms_recommend_outfitUrl.append(pair)
                except:
                    continue
        elif outfit_info['outfit_Bodypart'][outfit_info['outfit_Item']==m[0]].values[0]=='dress':
            if outfit_info['outfit_Bodypart'][outfit_info['outfit_Item']==j[0]].values[0] \
            in ['outerwear','acc']:
                try:
                    pair = (j[0],outfit_dict['summer',j[0]][:3] , j[1])
                    ms_recommend_outfitUrl.append(pair)   
                except:
                    continue
    return m_recommend_outfitUrl, ms_recommend_outfitUrl

In [42]:
similar_products_byitem_acc('summer',model['白色褲裝'])

(('白色褲裝',
  ['https://wear.tw/yoahiru35/17301668/',
   'https://wear.tw/fwafwa7/17455693/',
   'https://wear.tw/maarimo196/17383511/'],
  1.0),
 [('綠色手鏈',
   ['https://wear.tw/tomoka0125/14765320/',
    'https://wear.tw/tomoka0125/14771087/'],
   0.6203427910804749),
  ('粉色戒指',
   ['https://wear.tw/moimaru6/17148539/',
    'https://wear.tw/choco24/7405827/',
    'https://wear.tw/unakmkmay/17279229/'],
   0.6091607809066772),
  ('綠色錢包', ['https://wear.tw/yoahiru35/15127493/'], 0.6048542261123657),
  ('黃色背心外套', ['https://wear.tw/maarimo196/17265532/'], 0.595501720905304),
  ('橘色手袋',
   ['https://wear.tw/0627himari/17000746/',
    'https://wear.tw/0627himari/17133203/',
    'https://wear.tw/0627himari/17307512/'],
   0.5877591371536255),
  ('棕色发夹',
   ['https://wear.tw/xxyukie522xx/17100979/',
    'https://wear.tw/xxyukie522xx/17520767/',
    'https://wear.tw/xxyukie522xx/17337942/'],
   0.5831711888313293),
  ('粉色頭飾',
   ['https://wear.tw/moimaru6/17010919/',
    'https://wear.tw/yoahiru